# Extract & Transform

In [6]:
from PublicDataReader import Kbland
import pandas as pd
import numpy as np

api = Kbland()

def make_price_index():
    """
    kB부동산 매매가격지수 모든 유형별 단일 테이블화 
    
    building_category
        '01':'아파트',
        '08':'연립',
        '09':'단독',
        '98':'주택종합'
        
    trade_category
        '01':'매매',
        '02':'전세'
        
    """
    building_category = ['01','08','09','98']
    trade_category = ['01','02']

    df_price_index = pd.DataFrame()
    itercount = 0

    for i in building_category:
        for j in trade_category:
            params = {
                "월간주간구분코드": "01",
                "매물종별구분": i,
                "매매전세코드": j,
                "지역코드": "",
                "기간": "",
            }
            df = api.get_price_index(**params)

            # api별로 컬럼명 변경을 위한 단어추출
            first_word = df[df.columns[:3]].iloc[0].values[0]
            second_word = df[df.columns[:3]].iloc[0].values[1]
            third_word = df[df.columns[:3]].iloc[0].values[2]
            last_word = df.columns[-1]
            new_col_name = first_word + ' ' + second_word + ' ' + third_word + ' ' + last_word

            # 지수컬럼명 변경
            df = df.rename(columns = {
                        last_word : new_col_name
                    })

            # 필요한 컬럼만 남기기
            df = df[df.columns[3:]]

            if itercount == 0:
                df_price_index = pd.concat([df_price_index, df])
            else:
                df_price_index = pd.merge(df_price_index, df, on = ['지역코드','지역명','날짜'], how = 'left')

            itercount += 1
    return df_price_index

In [7]:
df = make_price_index()

# Load to GBQ

In [1]:
import sys
import os
from pathlib import Path
path = 'C:\\Users\\ljs74\\Documents\\GitHub\\Personal-Data-Lake\\database'
sys.path.append(path)
sys.path.append(str(Path(os.getcwd())))

In [3]:
from gcp import BigqueryHandler

In [5]:
BH = BigqueryHandler()
BH.read_table(df)

TypeError: read_table() missing 1 required positional argument: 'sql'